In [42]:
Sent1 = "Calculated solution molecular masses are 213 kDa for PROTEIN, 73 kDa for PROTEIN1, and 186 kDa for both."

In [43]:
Sent2 = "The recombinant material is similar to authentic PROTEIN from Acanthamoeba-based on fluorescence monitored urea denaturation, circular dichroism, actin-nucleotide exchange rate and the Kd for rabbit PROTEIN1."

In [44]:
import os
import json
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import pandas as pd
import scipy
import re
import logging
import random
import numpy as np
import pandas as pd
import transformers
import warnings
import torch
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pratik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [45]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
config = AutoConfig.from_pretrained("https://s3.amazonaws.com/models.huggingface.co/bert/monologg/biobert_v1.1_pubmed/config.json")
tokenizer = AutoTokenizer.from_pretrained("monologg/biobert_v1.1_pubmed")
model = AutoModel.from_pretrained("monologg/biobert_v1.1_pubmed",config=config)
max_len= 512

In [46]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (device)
model.to(device)
model.eval()

cpu


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [47]:
from nltk import sent_tokenize
stopwords=['introduction','document','begin','end','amssymb','documentclass','wasysym','oddsidemargin','12pt','69pt','minimal','mathrsfs','amsbsy','background','abstract','title:','introduction:','background:','abstract:','title','doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
    'rights', 'reserved','conclusion', 'permission', 'used','al.', 'using', 'biorxiv', 'medrxiv', 'license', 'Fig', 'Fig.','Figs','Figs.','fig', 'fig.', 'et','al','day',
    'al.', 'elsevier', 'PMC', 'CZI', 'www','com','abstract:','introduction','that','because','of','title','abstract','cite',"Wang","may",
    "Devi","Zhang","Li","Abstract","Liu","Singh","Yang","Kumar","Wu","Xu","Smith","Johnson","Williams","Brown","Jones","Miller","Davis","Wilson", "Anderson", "Taylor","usepackage","amsfonts","amsmath","amsby","setlength","upgreek"]

In [48]:
len(numbers)

122

In [49]:
clean = re.sub(r"""
               [,@#;:?!()&$]+  # Accept one or more copies of punctuation
               \ *           # plus zero or more copies of a space,
               """,
               " ",          # and replace it with a single space
               Sent1, flags=re.VERBOSE)

In [50]:
clean

'Calculated solution molecular masses are 213 kDa for PROTEIN 73 kDa for PROTEIN1 and 186 kDa for both.'

In [51]:
words=clean.split()
print (words)
resultwords= [word for word in words if word.lower() not in stopwords and word not in numbers]

['Calculated', 'solution', 'molecular', 'masses', 'are', '213', 'kDa', 'for', 'PROTEIN', '73', 'kDa', 'for', 'PROTEIN1', 'and', '186', 'kDa', 'for', 'both.']


In [52]:
resultwords

['Calculated',
 'solution',
 'molecular',
 'masses',
 'are',
 '213',
 'kDa',
 'for',
 'PROTEIN',
 '73',
 'kDa',
 'for',
 'PROTEIN1',
 'and',
 '186',
 'kDa',
 'for',
 'both.']

In [53]:
text = ' '.join([i for i in resultwords])
# Create sentence tokens
ls = sent_tokenize(text)
l=[s for s in ls if len(s)>=50]
tokenized = [tokenizer.encode(x, add_special_tokens=True,truncation=True) for x in l]
padded = np.array([x + [0]*(max_len-len(x)) for x in tokenized])
padded

array([[  101, 11917, 24029,  1181,  5072,  9546, 12980,  1132, 21640,
          180,  2137,  1161,  1111, 11629, 14697, 27514,  2249,  5766,
          180,  2137,  1161,  1111, 11629, 14697, 27514,  2249,  1475,
         1105, 21331,   180,  2137,  1161,  1111,  1241,   119,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

## Creation of the token embeddings

In [58]:
attention_mask = np.where(padded != 0, 1, 0)

In [59]:
input_ids = torch.tensor(padded).to(torch.int64).to(device)
attention_mask = torch.tensor(attention_mask).to(torch.int64).to(device)
print (input_ids.size(), attention_mask.size())

torch.Size([1, 512]) torch.Size([1, 512])


In [60]:
with torch.no_grad():
    outputs, cls = model(input_ids, attention_mask=attention_mask)

In [61]:
print (outputs.shape, cls.shape)

torch.Size([1, 512, 768]) torch.Size([1, 768])


## BioBERT embedding of a sentence

In [62]:
cls

tensor([[-5.0766e-02,  2.6324e-02,  9.6780e-01, -1.0000e+00,  9.9999e-01,
          1.3029e-01,  1.6034e-01,  9.4219e-01, -5.4331e-02,  6.8003e-02,
          8.4922e-01,  9.9992e-01, -1.1338e-02, -8.8088e-01, -6.8710e-02,
          8.0648e-03,  1.0000e+00,  4.1214e-02, -9.9626e-01, -1.4326e-01,
          7.0472e-02, -9.7107e-01, -3.0532e-02,  9.9108e-01, -1.0616e-01,
         -5.8385e-02,  9.9988e-01,  9.9624e-01, -1.7464e-02, -1.9913e-01,
          1.3201e-02, -1.0000e+00,  9.9549e-01, -9.9993e-01,  8.3575e-02,
          1.3578e-02, -3.6289e-02,  6.6727e-02,  7.9160e-01, -9.9875e-01,
          1.0049e-01, -5.9257e-01,  8.5778e-02,  3.7033e-02,  9.9898e-01,
          5.1273e-03, -1.4944e-02, -8.1968e-03,  1.8911e-02,  3.4958e-01,
          1.5714e-01,  9.7260e-01,  6.5083e-01,  9.9824e-01,  9.9967e-01,
         -8.7032e-02,  1.0000e+00,  5.6373e-02,  9.0991e-01, -1.1657e-01,
          9.9999e-01, -1.8076e-02, -6.5881e-02,  1.8510e-01,  8.9966e-02,
         -1.0804e-01, -8.5976e-01, -9.